In [1]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau

path = 'Rice_Leaf_AUG'

train_dataset = image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32  
)

validation_dataset = image_dataset_from_directory(
    path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

# Store class_names before applying operations
class_names = train_dataset.class_names

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',  # Monitor the validation loss to adjust the learning rate
    factor=0.5,          # Reduce the learning rate by a factor of 0.5
    patience=3,          # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=1e-6,         # Lower bound on the learning rate
    verbose=1            # Print updates about learning rate reduction
)

model = tf.keras.models.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=25,
    callbacks=[lr_scheduler] 
)

model.evaluate(validation_dataset, verbose=2)

model.export('rice_final')

Found 3829 files belonging to 6 classes.
Using 3064 files for training.
Found 3829 files belonging to 6 classes.
Using 765 files for validation.


2024-09-15 08:59:42.803727: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Max
2024-09-15 08:59:42.803746: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2024-09-15 08:59:42.803760: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2024-09-15 08:59:42.803773: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-15 08:59:42.803782: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/25


2024-09-15 08:59:43.348941: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


96/96 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step - accuracy: 0.2431 - loss: 1.9382 - val_accuracy: 0.4340 - val_loss: 1.3661 - learning_rate: 0.0010
Epoch 2/25
96/96 ━━━━━━━━━━━━━━━━━━━━ 6s 59ms/step - accuracy: 0.4803 - loss: 1.3175 - val_accuracy: 0.5595 - val_loss: 1.1159 - learning_rate: 0.0010
Epoch 3/25
96/96 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - accuracy: 0.6032 - loss: 1.0979 - val_accuracy: 0.6118 - val_loss: 1.0124 - learning_rate: 0.0010
Epoch 4/25
96/96 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - accuracy: 0.6362 - loss: 0.9516 - val_accuracy: 0.6392 - val_loss: 0.8888 - learning_rate: 0.0010
Epoch 5/25
96/96 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - accuracy: 0.6847 - loss: 0.8267 - val_accuracy: 0.6641 - val_loss: 0.8520 - learning_rate: 0.0010
Epoch 6/25
96/96 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - accuracy: 0.7575 - loss: 0.6432 - val_accuracy: 0.6784 - val_loss: 0.9203 - learning_rate: 0.0010
Epoch 7/25
96/96 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - accuracy: 0.7923 - loss: 0.5765 - val_accuracy: 0.70

INFO:tensorflow:Assets written to: rice_final/assets


Saved artifact at 'rice_final'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 6), dtype=tf.float32, name=None)
Captures:
  6165488528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6165489488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6165489872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6165490832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6165491216: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6165492176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6165493136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6165494096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6165493520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6165494864: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [2]:
import openvino as ov
from openvino.tools import mo
from openvino.runtime import serialize

# Path to your SavedModel directory
saved_model_path = "rice_final"

ov_model = mo.convert_model(saved_model_path, 
                            model_name="rice_model",
                            framework="tf", input_shape=[1, 224, 224, 3])  

# Specify the output directory and filenames
output_dir = "openvino_model"
xml_filename = "new_rice_model.xml"
bin_filename = "new_rice_model.bin"

# Serialize the model to IR format
serialize(ov_model, xml_filename, bin_filename)

print(f"Model converted and saved to {output_dir}/{xml_filename} and {output_dir}/{bin_filename}")

[ INFO ] MO command line tool is considered as the legacy conversion API as of OpenVINO 2023.2 release.
In 2025.0 MO command line tool and openvino.tools.mo.convert_model() will be removed. Please use OpenVINO Model Converter (OVC) or openvino.convert_model(). OVC represents a lightweight alternative of MO and provides simplified model conversion API. 
Find more information about transition from MO to OVC at https://docs.openvino.ai/2023.2/openvino_docs_OV_Converter_UG_prepare_model_convert_model_MO_OVC_transition.html
Model converted and saved to openvino_model/new_rice_model.xml and openvino_model/new_rice_model.bin


In [ ]:
import requests

# parametros primera recomendacion y segunda
crop = "Cucumber"
disease = "Bacterial Wilt"

recommendation1 = requests.post("https://fridaplatform.com/generate", json={"inputs": f"Provea 2 pequeñas observaciones de la planta {crop} a cuál crece actualmente en un invernadero con el estado de la planta predominante: (English state just for reference, yet answer in Spanish){disease}", "parameters": {"max_new_tokens": 400, "stream": True}})


print(recommendation1.json()['generated_text'])

# parametros de la segunda recomendacion
first_state = "Healthy Leaf"
first_percentage = 60
second_state = "Blight Leaf"
second_percentage = 30
third_state = "Dark Leaf"
third_percentage = 10


recommendation2 = requests.post("https://fridaplatform.com/generate", json={"inputs": f"Provea un calificación del 1 al 10 de en qué tan buen estado se encuentra un invernadero dado las siguientes estadísticas y estado de la planta: %{first_percentage} de la planta {crop} se encuentra como[utilizar nombre del estado en español para responder] {first_state}, %{second_percentage} se encuentra en la condición de {second_state}, y %{third_percentage} se encuentra en la condición de {third_state}. [Presente la respuesta como un único pequeño párrafo]", "parameters": {"max_new_tokens": 300, "stream": True}})

print(recommendation2.json()['generated_text'])

# curl -X POST -H "Content-Type: application/json" -d '{"inputs":"Help my friend write a python function for tensorflow model training", "parameters":{"max_new_tokens":300, "stream":true}}' https://fridaplatform.com/generate